# Machine Learning

## Session #7: Hidden Markov Models for stock data analysis

This notebook is based on the notebook [``plot_hmm_stock_analysis.ipynb``](http://hmmlearn.readthedocs.org/en/stable/auto_examples/plot_hmm_stock_analysis.html#sphx-glr-auto-examples-plot-hmm-stock-analysis-py) of [hmmlearn](http://hmmlearn.readthedocs.org/en/stable/index.html) and [``date_demo1.py``](http://matplotlib.org/examples/pylab_examples/date_demo1.html) of [matplotlib](http://matplotlib.org/index.html).



### 1. Yahoo! stock market time series

This script load a time series containing the stock price data from Yahoo! finance and shows how to use Gaussian HMM on it

In [ ]:
%matplotlib inline
from __future__ import print_function

import datetime

import numpy as np
from matplotlib import cm, pyplot as plt
from matplotlib.dates import YearLocator, MonthLocator
try:
    from matplotlib.finance import quotes_historical_yahoo_ochl
except ImportError:
    # For Matplotlib prior to 1.5.
    from matplotlib.finance import (
        quotes_historical_yahoo as quotes_historical_yahoo_ochl
    )

from hmmlearn.hmm import GaussianHMM

import hmmlearn.hmm as hmm
print(__doc__)

## Prepare the data 

In [ ]:
quotes = quotes_historical_yahoo_ochl(
    "INTC", datetime.date(1995, 1, 1), datetime.date(2015, 1, 6))

# Unpack quotes
dates = np.array([q[0] for q in quotes], dtype=int)
close_v = np.array([q[2] for q in quotes])
volume = np.array([q[5] for q in quotes])[1:]

# Take diff of close value. Note that this makes
# ``len(diff) = len(close_t) - 1``, therefore, other quantities also
# need to be shifted by 1.
diff = np.diff(close_v)
dates = dates[1:]
close_v = close_v[1:]

# Pack diff and volume for training.
X = np.column_stack([diff, volume])

In [ ]:
plt.plot(dates,X[:,0])
plt.show()
plt.plot(dates,X[:,1])
plt.show()

## Run Gaussian HMM


In [ ]:
print("fitting to HMM and decoding ...", end="")

# Make an HMM instance and execute fit
model = GaussianHMM(n_components=4, covariance_type="diag", n_iter=1000).fit(X)

# Predict the optimal sequence of internal hidden state
hidden_states = model.predict(X)

print("done")

## Print trained parameters and plot


In [ ]:
print("Transition matrix")
print(np.round(model.transmat_,2))
print()

print("Means and vars of each hidden state")
for i in range(model.n_components):
    print("{0}th hidden state".format(i))
    print("mean = ", np.round(model.means_[i],2))
    print("var = ", np.round(np.diag(model.covars_[i]),2))
    print()

fig, axs = plt.subplots(model.n_components, sharex=True, sharey=True)
colours = cm.rainbow(np.linspace(0, 1, model.n_components))
for i, (ax, colour) in enumerate(zip(axs, colours)):
    # Use fancy indexing to plot data in each state.
    mask = hidden_states == i
    ax.plot_date(dates[mask], close_v[mask], ".-", c=colour)
    ax.set_title("{0}th hidden state".format(i))

    # Format the ticks.
    ax.xaxis.set_major_locator(YearLocator())
    ax.xaxis.set_minor_locator(MonthLocator())

    ax.grid(True)

plt.show()

### 2. Changing the model parameters

* Change the number of hidden states, from 2 to 6, calculate the likelihood of each model. Interpret the results.
* With 4 hidden states, change the number of initizalizations to 1 and evaluate the likelihood of the model in 10 runs of the EM algorithm


### 3. Changing the model

Change the emission probability to a GMMHMM, and repeat #2 varying the number of component from 1 to 3 and the number of hidden states from 2 to 5